# Basic

In [6]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [16]:
import torch

print(torch.__version__)
print(torch.backends.mps.is_built()) # Check whether mps is built or not
print(torch.backends.mps.is_available()) # Check availability of mps


1.13.1
True
True


In [ ]:
"""
mps_device = torch.device("mps")

# MPS 장치에 바로 tensor를 생성합니다.
x = torch.ones(5, device=mps_device)
# 또는
x = torch.ones(5, device="mps")

# GPU 상에서 연산을 진행합니다.
y = x * 2

# 또는, 다른 장치와 마찬가지로 MPS로 이동할 수도 있습니다.
model = YourFavoriteNet()  # 어떤 모델의 객체를 생성한 뒤,
model.to(mps_device)       # MPS 장치로 이동합니다.

# 이제 모델과 텐서를 호출하면 GPU에서 연산이 이뤄집니다.
pred = model(x)
"""

In [8]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [10]:
# Load Training Dataset from Open Source
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

# Load Test Dataset from Open Source
test_data = datasets.MNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [11]:
batch_size=64

# Create DataLoader
# tf.data.Dataset
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of x [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of x [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [18]:
device = torch.device("mps")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
                nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Dropout(.2),
            nn.Linear(128,10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [22]:
# Setting Loss function and Optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [21]:
# Training Function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute Predicted error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # BackPropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [30]:
# Test Function
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    # Setting no gradient
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>.1f}%, Avg loss: {test_loss:>8f} \n")

In [31]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n{'-'*30}")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
------------------------------
loss: 0.063536 [    0/60000]
loss: 0.097155 [ 6400/60000]
loss: 0.067168 [12800/60000]
loss: 0.056007 [19200/60000]
loss: 0.083446 [25600/60000]
loss: 0.113837 [32000/60000]
loss: 0.032853 [38400/60000]
loss: 0.119776 [44800/60000]
loss: 0.102493 [51200/60000]
loss: 0.122135 [57600/60000]
Test Error: 
 Accuracy: 96.8%, Avg loss: 0.102825 

Epoch 2
------------------------------
loss: 0.044799 [    0/60000]
loss: 0.074989 [ 6400/60000]
loss: 0.057560 [12800/60000]
loss: 0.043096 [19200/60000]
loss: 0.063139 [25600/60000]
loss: 0.082683 [32000/60000]
loss: 0.022273 [38400/60000]
loss: 0.086121 [44800/60000]
loss: 0.088209 [51200/60000]
loss: 0.080135 [57600/60000]
Test Error: 
 Accuracy: 97.2%, Avg loss: 0.091518 

Epoch 3
------------------------------
loss: 0.031103 [    0/60000]
loss: 0.065467 [ 6400/60000]
loss: 0.047104 [12800/60000]
loss: 0.037377 [19200/60000]
loss: 0.045875 [25600/60000]
loss: 0.059825 [32000/60000]
loss: 0.018427 [38400/600

# Tensor

In [32]:
# list -> tensor
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)
print(x_data)

tensor([[1, 2],
        [3, 4]])


In [34]:
# numpy array -> tensor

#torch.tensor: copy
np_array = np.array(data)
x_np_1 = torch.tensor(np_array)
print(x_np_1)

#torch.as_tensor: view
x_np_2 = torch.as_tensor(np_array)
print(x_np_2)

#torch.from_numpy: view
x_np_3 = torch.from_numpy(np_array)
print(x_np_3)

tensor([[1, 2],
        [3, 4]])
tensor([[1, 2],
        [3, 4]])
tensor([[1, 2],
        [3, 4]])


In [35]:
x_np_1[0,0] = 5
print(x_np_1)
print(np_array)

tensor([[5, 2],
        [3, 4]])
[[1 2]
 [3 4]]


In [38]:
x_np_2[0,0] = 6
print(x_np_2)
print(np_array)
print("-"*25)
x_np_3[0,0] = 7
print(x_np_3)
print(np_array)


tensor([[6, 2],
        [3, 4]])
[[6 2]
 [3 4]]
-------------------------
tensor([[7, 2],
        [3, 4]])
[[7 2]
 [3 4]]


In [39]:
np_again = x_np_1.numpy()
print(np_again, type(np_again))

[[5 2]
 [3 4]] <class 'numpy.ndarray'>


In [40]:
a = torch.ones(2,3)
b = torch.zeros(2,3)
c = torch.full((2,3), 2)
d = torch.empty(2,3)
print(a)
print(b)
print(c)
print(d)

tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[2, 2, 2],
        [2, 2, 2]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [42]:
e = torch.zeros_like(c)
f = torch.ones_like(c)
g = torch.full_like(c, 3)
h = torch.empty_like(c)
print(e)
print(f)
print(g)
print(h)

tensor([[0, 0, 0],
        [0, 0, 0]])
tensor([[1, 1, 1],
        [1, 1, 1]])
tensor([[3, 3, 3],
        [3, 3, 3]])
tensor([[0, 0, 0],
        [0, 0, 0]])


In [43]:
i = torch.eye(3)
print(i)

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])


In [44]:
j = torch.arange(10)
print(j)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [45]:
k = torch.rand(2,2)
l = torch.randn(2,2)
print(k)
print(l)

tensor([[0.8748, 0.8557],
        [0.6798, 0.3661]])
tensor([[-0.0574,  0.6550],
        [ 0.2733,  1.6249]])


In [46]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


In [47]:
tensor = tensor.reshape(4,3)
tensor = tensor.int()
if torch.backends.mps.is_available():
    tensor = tensor.to('mps')
print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([4, 3])
Datatype of tensor: torch.int32
Device tensor is stored on: mps:0


In [48]:
a = torch.arange(1,13).reshape(3,4)
print(a)

tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]])


In [49]:
# indexing
print(a[1])
print(a[0,-1])

tensor([5, 6, 7, 8])
tensor(4)


In [50]:
# slicing
print(a[1:-1])
print(a[:2, 2:])

tensor([[5, 6, 7, 8]])
tensor([[3, 4],
        [7, 8]])


### Transpose

In [51]:
a = torch.arange(16).reshape(2,2,4)
print(a, a.shape)

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7]],

        [[ 8,  9, 10, 11],
         [12, 13, 14, 15]]]) torch.Size([2, 2, 4])


In [52]:
b = a.transpose(1, 2)
print(b, b.shape)

tensor([[[ 0,  4],
         [ 1,  5],
         [ 2,  6],
         [ 3,  7]],

        [[ 8, 12],
         [ 9, 13],
         [10, 14],
         [11, 15]]]) torch.Size([2, 4, 2])


In [54]:
c = a.permute((2, 0, 1))
print(c, c.shape)

tensor([[[ 0,  4],
         [ 8, 12]],

        [[ 1,  5],
         [ 9, 13]],

        [[ 2,  6],
         [10, 14]],

        [[ 3,  7],
         [11, 15]]]) torch.Size([4, 2, 2])


In [55]:
x = torch.tensor([[1,2],[3,4]], dtype=torch.float32)
y = torch.tensor([[5,6],[7,8]], dtype=torch.float32)

In [56]:
# in-place 연산
print(x.add(y))
print(x)
# _를 붙이면 inplace=True와 같아짐
print(x.add_(y))
print(x)

tensor([[ 6.,  8.],
        [10., 12.]])
tensor([[1., 2.],
        [3., 4.]])
tensor([[ 6.,  8.],
        [10., 12.]])
tensor([[ 6.,  8.],
        [10., 12.]])


In [57]:
z = torch.arange(1,11).reshape(2,5)
print(z)

tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10]])


In [59]:
sum1 = torch.sum(z, axis=0)
sum2 = torch.sum(z, axis=1)
sum3 = torch.sum(z, axis=-1)
print(sum1, sum1.shape)
print(sum2, sum2.shape)
print(sum3, sum3.shape)

tensor([ 7,  9, 11, 13, 15]) torch.Size([5])
tensor([15, 40]) torch.Size([2])
tensor([15, 40]) torch.Size([2])


In [62]:
a = torch.arange(24).reshape(4,6)
# a -copy-> b
b = a.clone().detach()
print(a, a.shape)
print(b, b.shape)

tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23]]) torch.Size([4, 6])
tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23]]) torch.Size([4, 6])


In [63]:
c = torch.cat([a,b], axis=0)
print(c, c.shape)

tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23],
        [ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23]]) torch.Size([8, 6])


In [64]:
d = torch.cat([a,b], axis=1)
print(d, d.shape)

tensor([[ 0,  1,  2,  3,  4,  5,  0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11,  6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17, 12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23, 18, 19, 20, 21, 22, 23]]) torch.Size([4, 12])


In [65]:
e = torch.stack([a,b], axis=0)
print(e, e.shape)

tensor([[[ 0,  1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10, 11],
         [12, 13, 14, 15, 16, 17],
         [18, 19, 20, 21, 22, 23]],

        [[ 0,  1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10, 11],
         [12, 13, 14, 15, 16, 17],
         [18, 19, 20, 21, 22, 23]]]) torch.Size([2, 4, 6])


In [66]:
f = torch.stack([a,b], axis=1)
print(f, f.shape)

tensor([[[ 0,  1,  2,  3,  4,  5],
         [ 0,  1,  2,  3,  4,  5]],

        [[ 6,  7,  8,  9, 10, 11],
         [ 6,  7,  8,  9, 10, 11]],

        [[12, 13, 14, 15, 16, 17],
         [12, 13, 14, 15, 16, 17]],

        [[18, 19, 20, 21, 22, 23],
         [18, 19, 20, 21, 22, 23]]]) torch.Size([4, 2, 6])


In [67]:
g = torch.stack([a,b], axis=-1)
print(g, g.shape)

tensor([[[ 0,  0],
         [ 1,  1],
         [ 2,  2],
         [ 3,  3],
         [ 4,  4],
         [ 5,  5]],

        [[ 6,  6],
         [ 7,  7],
         [ 8,  8],
         [ 9,  9],
         [10, 10],
         [11, 11]],

        [[12, 12],
         [13, 13],
         [14, 14],
         [15, 15],
         [16, 16],
         [17, 17]],

        [[18, 18],
         [19, 19],
         [20, 20],
         [21, 21],
         [22, 22],
         [23, 23]]]) torch.Size([4, 6, 2])
